# Replicating the Simulation Results from the Published Paper

Checking whether the results from the Beisbart, Betz, and Brun (2021) (generated with an implementation of the model in Mathematica, see [here](https://github.com/debatelab/remoma)) are replicable in the current Python implementation.

**Result:** The results from Beisbart, Betz, and Brun (2021) can be reproduced with the Python model. 

*Remark:* Beisbart, Betz, and Brun (2021) do not specify standard deviations of their results. We omit these as well for simplicity and regard the results as sufficiently similar. (Admittedly, an uncertainty analysis should corroborate this hypothesis.)  

## How to run this notebook

There are several possibilities to execute this notebook. You can, for instance,

1. execute this notebook on Colab: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/re-models/re-technical-report/blob/main/notebooks/replication_bbb_2021.ipynb), or
2. execute this notebook locally in, for instance, [JupyterLab](https://jupyter.org/) by cloning the Github repository of the report (for details, see <https://github.com/re-models/re-technical-report>).

## Installing and importing libraries

In [ ]:
!pip install re-technical-report

In [2]:
from theodias import StandardPosition, DAGDialecticalStructure
from rethon import StandardGlobalReflectiveEquilibrium
from theodias.util import random_position_as_set
import random
from re_technical_report import random_weights

from IPython.display import clear_output

In [3]:
# The standard example (see Beisbart, Betz, and Brun 2021, 451):
dia = DAGDialecticalStructure(7, initial_arguments=[[1,3],[1,4],[1,5],[1,-6],[2,-4],[2,5],[2,6],[2,7]])

pos1 = StandardPosition.from_set({3,4,5}, 7)
pos2 = StandardPosition.from_set({2,3,4,5}, 7)
pos3 = StandardPosition.from_set({3,4,5,6,7}, 7)
pos4 = StandardPosition.from_set({3,4,5,-6,7}, 7)

positions = [pos1, pos2, pos3, pos4]

re = StandardGlobalReflectiveEquilibrium(dia)
print(re.model_parameters()['weights'])

{'account': 0.35, 'systematicity': 0.55, 'faithfulness': 0.1}


## First Ensemble

- Dialectical structure: standard example
- 500 randomly generated initial commitments
- Fixed weights: account = 0.35, systematicity = 0.55, faithfulness = 0.1

> The first ensemble is based on a random sample of initial conditions (N = 500) on the basic dialectical
structure defined above. Using the same values of the weights as before, we find
that in 95% of all cases, the equilibration fixed points are also global optima. Of
these, 75% are full RE states (Beisbart, Betz, and Brun 2021, 455).

In [4]:
n = 500

go_count = 0
full_re_count = 0

for i in range(n):
    clear_output(wait=True)
    print('Progress [%]: ', round((i/n)*100,1))
    init_coms = StandardPosition.from_set(random_position_as_set(7, allow_empty_position=False), 7)
    re.set_initial_state(init_coms)
    re.re_process()
    tcp = (re.state().last_theory(), re.state().last_commitments())
    global_optima = re.global_optima(init_coms)
    
    if tcp in global_optima:
        go_count += 1
        
        if re.dialectical_structure().closure(tcp[0]) == tcp[1]:
            full_re_count += 1
            
clear_output(wait=True)


print("Total: ", n)
print("Number of reached global optima: ", go_count)
print("Relative to total [%]", round((go_count/n)*100, 2))
print("Number of full RE states among reached global optima: ", full_re_count)
print("Relative to global optima [%]", round((full_re_count/go_count)*100, 2))

Total:  500
Number of reached global optima:  468
Relative to total [%] 93.6
Number of full RE states among reached global optima:  366
Relative to global optima [%] 78.21


## Second Ensemble

- Dialectical structure: standard example
- 4 standard initial commitments
- Randomly generated weights

> The second ensemble (N = 500) uses the initial commitments and dialectical structure from the four illustrative cases discussed above, but randomly varies the weights within the achievement function. Given such systematic parameter perturbation, 88% of the equilibration fixed points are also global optima; and of these, 65% are full RE states (Beisbart, Betz, and Brun 2021, 455).

In [10]:
n = 125

go_count = 0
full_re_count = 0

for i in range(n):
    
    a,f,s = random_weights()
    re.model_parameters()["weights"] = {"account":a, "systematicity":s, "faithfulness":f}
    clear_output(wait=True)
    print('Progress [%]: ', round((i/n)*100,1))
          
    for pos in positions:
        re.set_initial_state(pos)
        re.re_process()
        tcp = (re.state().last_theory(), re.state().last_commitments())
        global_optima = re.global_optima(pos)
        
        if tcp in global_optima:
            go_count += 1
            if re.dialectical_structure().closure(tcp[0]) == tcp[1]:
                full_re_count += 1

clear_output(wait=True)
print("Total number of model runs: ", n*4)
print("Number of reached global optima: ", go_count)
print("Relative to total [%]", round((go_count/(n*4))*100, 2))
print("Number of full RE states among reached global optima: ", full_re_count)
print("Relative to global optima [%]", round((full_re_count/go_count)*100, 2))

Total number of model runs:  500
Number of reached global optima:  439
Relative to total [%] 87.8
Number of full RE states among reached global optima:  307
Relative to global optima [%] 69.93
